# <center> Naive Bayes:  fetch20_newsgroups </center>
<center> University of Denver </center>
<center> Eric Browne </center>

# Uploading the Data

In [ ]:
## imported libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import *
from sklearn.metrics import *
from math import sqrt
from math import pi
from math import exp

import nltk
from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

twenty_train.target_names

In [ ]:
print(len(twenty_train.data))
print(len(twenty_train.filenames))

In [ ]:
## print some test lines:
print("\n".join(twenty_train.data[0].split("\n")[:3]))
print(twenty_train.target_names[twenty_train.target[0]])

In [ ]:
## Start count Vectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

# Data Preprocessing 

In [ ]:
count_vect.vocabulary_.get(u'algorithm')

# Data Splitting 

In [ ]:
## TF-IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

# Model Building 

In [ ]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

# Model Evaluation 

In [ ]:
## Testing on some new data:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

In [ ]:
## Actually building a pipeline:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

In [ ]:
text_clf.get_params()

In [ ]:
text_clf.fit(twenty_train.data, twenty_train.target)

In [ ]:
## Doing a test evaluation:
twenty_test = fetch_20newsgroups(subset='test',categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

In [ ]:
## Grid Search:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf, parameters, cv=3)

In [ ]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

In [ ]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

In [ ]:
gs_clf.best_score_

In [ ]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))


# Conclusion 

**The best hyper parameters for the model were displayed above.  I do not believe that the model overfit, as even with the new test set, the accuracy was still 93%.  The purpose of this mini project was to distinguish lines of texts between either being religious/non religious, and in those two sections, being either atheist, christian, about computer graphics, or about science and medicine.  For the model, I used CountVectorization and a TDIDFTransformer, paired with a Gaussian Naive Bayes to classify each line of text.  The data was taken from one of the sklean imported datasets about news groups.  This may come in handy when trying to classify whether or not a piece of news is relating to a certain topic.**